In [ ]:
import logging
import pickle
import time
from collections import Counter
from concurrent.futures import ProcessPoolExecutor, as_completed
from pathlib import Path
from typing import Sequence

import numpy as np
from numpy.typing import NDArray

from farkle.simulation import (
    _play_game,
    generate_strategy_grid,  # <- replace with your real loader
)
from farkle.strategies import ThresholdStrategy

# ── Configuration constants ──────────────────────────────────────────────────
GAMES_PER_STRAT = 10_223
DESIRED_SEC_PER_CHUNK = 10

# ── Throughput estimation ────────────────────────────────────────────────────
def measure_throughput(
    strategies: Sequence[ThresholdStrategy],
    test_games: int = 2000,
    seed: int = 0
) -> float:
    """
    Simulate a small batch to estimate games/sec.
    """
    rng = np.random.default_rng(seed)
    seeds = rng.integers(0, 2**32 - 1, size=test_games)
    t0 = time.perf_counter()
    for s in seeds:
        _play_game(s, strategies)
    dt = time.perf_counter() - t0
    print(test_games / dt)
    return test_games / dt

def compute_chunk_size(
    strategies: Sequence[ThresholdStrategy],
    desired_sec: int = DESIRED_SEC_PER_CHUNK,
    min_chunk: int = 1000,
    max_chunk: int = 50000
) -> int:
    """
    Choose a chunk size so each batch takes about `desired_sec` seconds,
    clamped between min_chunk and max_chunk.
    """
    gps = measure_throughput(strategies)
    chunk = int(desired_sec * gps)
    print(max(min_chunk, min(chunk, max_chunk)))
    return max(min_chunk, min(chunk, max_chunk))

# ── Core simulation ──────────────────────────────────────────────────────────

def simulate_chunk(
    seeds: NDArray[np.int64],
    strategies: Sequence[ThresholdStrategy]
) -> Counter[str]:
    ctr: Counter[str] = Counter()
    for s in seeds.tolist():           # convert to Python int
        result = _play_game(int(s), strategies)
        ctr[result["winner"]] += 1
    return ctr

# ── Main entry point ─────────────────────────────────────────────────────────
def run_tournament(
    global_seed: int,
    checkpoint_path: str,
    n_jobs: int | None = None
) -> None:
    """
    1) Load strategies
    2) Auto-tune a chunk size
    3) Spawn workers to run all games in parallel, checkpointing every 10 chunks
    4) Load or finalize the win counter
    """
    # 1) load your strategies however you like
    strategies, metadata_df = generate_strategy_grid()   # e.g. load from YAML or generate_strategy_grid()
    n_strats = len(strategies)
    total_games = n_strats * GAMES_PER_STRAT

    # 2) figure out how big each chunk should be
    chunk_size = compute_chunk_size(strategies)

    # 3) prepare all the RNG seeds, split into chunks
    master_rng = np.random.default_rng(global_seed)
    all_seeds = master_rng.integers(0, 2**32 - 1, size=total_games)
    seed_chunks = [
        all_seeds[i : i + chunk_size]
        for i in range(0, total_games, chunk_size)
    ]

    # 4) set up logging & run
    logging.basicConfig(level=logging.INFO, format="%(asctime)s %(message)s")
    win_counter: Counter[str] = Counter()

    with ProcessPoolExecutor(max_workers=n_jobs) as exe:
        futures = {
            exe.submit(simulate_chunk, chunk, strategies): idx
            for idx, chunk in enumerate(seed_chunks)
        }
        for fut in as_completed(futures):
            idx = futures[fut]
            chunk_result = fut.result()
            win_counter.update(chunk_result)
            logging.info(
                f"Chunk {idx+1}/{len(seed_chunks)} done; "
                f"total wins={sum(win_counter.values())}"
            )

            # checkpoint every 10 chunks
            if (idx + 1) % 10 == 0:
                with open(checkpoint_path, "wb") as f:
                    pickle.dump(win_counter, f)

    # 5) final load or fallback
    chk = Path(checkpoint_path)
    if chk.exists():
        with chk.open("rb") as f:
            final_counter = pickle.load(f)
    else:
        final_counter = win_counter

    logging.info(f"Final win counts: {final_counter}")
    
    
def main(global_seed=0, checkpoint_path="checkpoint.pkl", n_jobs=16):
    global_seed = global_seed
    checkpoint_path=checkpoint_path
    n_jobs=n_jobs
    run_tournament(global_seed=0, checkpoint_path="checkpoint.pkl", n_jobs=16)

# ── Script guard ─────────────────────────────────────────────────────────────
if __name__ == "__main__":
    # Replace these with real arguments or argparse as you prefer
    main(global_seed=0, checkpoint_path="checkpoint.pkl", n_jobs=16)


In [ ]:
# Expected value of a single die roll is 25, of a turn is 150 (with no combos)

import numpy as np
rng_generator = np.random.default_rng()
rolls = []
for _ in range(1_000_000):
    roll = rng_generator.integers(1, 7, size=1)
    if roll == 5:
        rolls.append(50)
    elif roll == 1:
        rolls.append(100)
    else:
        rolls.append(0)
print(sum(rolls)/len(rolls))

25.00695
